### Data wrangling

In [1]:
#! pip install pycountry
#! pip install forex_python

In [2]:
import pandas as pd
import os
from Data_Preparation.PrepareData import *
from Data_Preparation.ScrapeWeb import *

In [3]:
# convert the csv file to pandas dataframe
df = pd.read_csv("Data/ks_dataset.csv")
# rename columns by removing extra space
df.rename(columns={col : col.strip().replace(" ","_") for col in list(df.columns)},inplace = True)
# check dataframe's information
df.info()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323750 entries, 0 to 323749
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             323750 non-null  int64  
 1   name           323746 non-null  object 
 2   category       323745 non-null  object 
 3   main_category  323750 non-null  object 
 4   currency       323750 non-null  object 
 5   deadline       323750 non-null  object 
 6   goal           323750 non-null  object 
 7   launched       323750 non-null  object 
 8   pledged        323750 non-null  object 
 9   state          323750 non-null  object 
 10  backers        323750 non-null  object 
 11  country        323750 non-null  object 
 12  usd_pledged    319960 non-null  object 
 13  Unnamed:_13    625 non-null     object 
 14  Unnamed:_14    12 non-null      object 
 15  Unnamed:_15    4 non-null       object 
 16  Unnamed:_16    1 non-null       float64
dtypes: float64(1), int64(1), obje

In [4]:
# check number of non missing values in each row
s = df.count(axis=1)
s.value_counts()

13    319336
12      3794
14       608
15         8
16         3
17         1
dtype: int64

##### Conclusion:
- We have 3794 rows (1.1%) with one missing value.
- 98,63% of rows have exactly 13 values


In [5]:
# instance of class that cleans the data
cleanData = CleanData(df)

#### Missing Values

In [6]:
# drop columns with mostly missing values
percent = 85
cleanData.missing_values(percent)

#### Basic Anomaly Detection

In [7]:
import os

# get notebook's folder directory
notebook_dir = os.path.dirname(os.path.abspath("solution.ipynb"))

# resolve inconsistency in data
# inconsistent data will be replaced by nan
params = dict()

params["numerical"] = [6,8,10,12]
params["dateTime"] = [5,7]

# dictionary of main_categories and corresponding categories
main_categories = select_options(notebook_dir,"main_category")
categories = select_options(notebook_dir,"category")
main_sub_dict = {main_categories[i]: categories[i]+[main_categories[i]] for i in range(len(main_categories))}

# string columns with known allowed values (flatten version of categories)
all_categories = [item for sublist in categories for item in sublist]

params["allowedValues"] = []
params["allowedValues"].append(
    (2,all_categories + main_categories))
params["allowedValues"].append(
    (3,main_categories))
params["allowedValues"].append(
    (4,select_options(notebook_dir,"currency")))
params["allowedValues"].append(
    (9,['failed', 'canceled', 'successful', 
        'live', 'undefined','suspended']))
params["allowedValues"].append(
    (11,select_options(notebook_dir,"country")))

cleanData.remove_basic_anomaly(params)

In [8]:
# delete rows with state "canceled","live","suspended"
df = df[df["state"].isin(["failed","successful","undefined",np.nan])]
df = df[df["category"].notna()]

In [9]:
df.state.value_counts()

failed        168221
successful    113081
undefined       3555
Name: state, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284857 entries, 0 to 323749
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             284857 non-null  int64         
 1   name           284854 non-null  object        
 2   category       284857 non-null  object        
 3   main_category  284857 non-null  object        
 4   currency       284857 non-null  object        
 5   deadline       284857 non-null  datetime64[ns]
 6   goal           284857 non-null  float64       
 7   launched       284857 non-null  datetime64[ns]
 8   pledged        284857 non-null  float64       
 9   state          284857 non-null  object        
 10  backers        284857 non-null  float64       
 11  country        281092 non-null  object        
 12  usd_pledged    281092 non-null  float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(6)
memory usage: 30.4+ MB


In [11]:
df.deadline.count()

284857

#### Advanced Anomaly Detection

In [12]:
# set cleanData dataframe attribute to the modified df
cleanData.df = df

# remove some advanced anomaly and fill nan values
cleanData.remove_advanced_anomaly(main_sub_dict)

In [13]:
df.to_csv("tidyData.csv")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284857 entries, 0 to 323749
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ID               284857 non-null  int64         
 1   name             284854 non-null  object        
 2   category         284857 non-null  object        
 3   main_category    284857 non-null  object        
 4   currency         284857 non-null  object        
 5   deadline         284857 non-null  datetime64[ns]
 6   goal             284857 non-null  float64       
 7   launched         284857 non-null  datetime64[ns]
 8   pledged          284857 non-null  float64       
 9   state            284857 non-null  object        
 10  backers          284857 non-null  float64       
 11  country          281092 non-null  object        
 12  usd_pledged      284857 non-null  float64       
 13  compaign_period  284857 non-null  float64       
dtypes: datetime64[ns](2)

In [18]:
# compaign periods can be from 1 day up to 60 days on kickstarter
# delete rows with compaign period greater than 60 days
df = df[df['compaign_period']<= (60*24*60*60)]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278492 entries, 0 to 323749
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ID               278492 non-null  int64         
 1   name             278489 non-null  object        
 2   category         278492 non-null  object        
 3   main_category    278492 non-null  object        
 4   currency         278492 non-null  object        
 5   deadline         278492 non-null  datetime64[ns]
 6   goal             278492 non-null  float64       
 7   launched         278492 non-null  datetime64[ns]
 8   pledged          278492 non-null  float64       
 9   state            278492 non-null  object        
 10  backers          278492 non-null  float64       
 11  country          274751 non-null  object        
 12  usd_pledged      278492 non-null  float64       
 13  compaign_period  278492 non-null  float64       
dtypes: datetime64[ns](2)

In [20]:
df.compaign_period.agg([min,max])

min      86400.0
max    5184000.0
Name: compaign_period, dtype: float64

In [21]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,compaign_period
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000.0,2015-08-11 12:12:28,0.0,failed,0.0,GB,0.0,5095412.0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000.0,2013-01-12 00:20:50,220.0,failed,3.0,US,220.0,3888000.0
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000.0,2012-03-17 03:24:11,1.0,failed,1.0,US,1.0,2595600.0
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000.0,2016-02-26 13:38:27,52375.0,successful,224.0,US,52375.0,3024000.0
5,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21 18:30:44,1000.0,2014-12-01 18:30:44,1205.0,successful,16.0,US,1205.0,1728000.0
